In [2]:
from pathlib import Path
import pyarrow.parquet as pq
from datasets import Dataset
import pandas as pd
from transformers import AutoProcessor, AutoConfig, AutoModelForCausalLM, Trainer, TrainingArguments
# from peft import LoraConfig, get_peft_model

import torch
import torch.nn as nn
import torchvision.transforms as T

from torch.utils.data import DataLoader
from safetensors.torch import load_file


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
cwd = str(Path.cwd())
filepath = cwd + "/datasets/train-00000-of-00738-80a58552f2fb3344.parquet"
model_name = "microsoft/phi-3.5-vision-instruct"
model_path = cwd + "/microsoft/phi-3.5-vision-instruct"
device = torch.device("mps" if torch.mps.is_available() else "cpu")
config = AutoConfig.from_pretrained(model_name)
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

NameError: name 'Path' is not defined

In [4]:
def preprocess(example):
    img = example.get("image")

    messages = [{
        "role": "user",
        "content": f"<|image_1|>\n{example['llm_generated_idea']}"
    }, {
        "role": "assistant",
        "content": example.get("text", "Okay")
    }]

    prompt = processor.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    inputs = processor(prompt, [img], return_tensors="pt")
    inputs = {k: v.squeeze(0) for k, v in inputs.items()}
    inputs["labels"] = inputs["input_ids"].clone()

    return inputs


In [6]:
model = AutoModelForCausalLM.from_config(config, attn_implementation="eager", trust_remote_code=True)

In [18]:
dataset = Dataset.from_parquet(filepath)
dataset = dataset.map(preprocess, remove_columns=["image", "llm_generated_idea", "text"])

Map: 100%|██████████| 2606/2606 [01:22<00:00, 31.76 examples/s]


In [ ]:
dataset[0]

In [ ]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)